In [9]:
import pandas as pd
import numpy as np
import os
import glob as gb
import sys
from decimal import Decimal
import datetime
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

sys.path.insert(0, "../../")
from clases.bd.conexion2 import MyDatabase2
conn = MyDatabase2()



Conexion exitosa con la Base de datos:irvin_hisminsa


In [20]:

def consulta_sql(conn):
 
    gestantes = conn.df("""
SELECT * FROM    (
        SELECT
            DISTINCT  
            mhtd.abrev_tipo_doc
            , mp.numero_documento
            , nt.id_paciente
            , nt.anio
            , nt.mes
            , nt.id_establecimiento
            , mhe.nombre_eess
            , nt.anio_actual_paciente AS edad_anio
            , ROUND(
                (
                    nt.fecha_atencion - mp.fecha_nacimiento
                )/ 30.44
                , 1
            ) AS edad_mes
            ,(
                nt.fecha_atencion - mp.fecha_nacimiento
            ) AS edad_dia
            , nt.fecha_atencion
            , nt.lote
            , nt.num_pag
            , nt.num_reg
            , nt.codigo_item
            , nt.talla
            , nt.peso
            , nt.peso_pregestacional
            , nt.fecha_ultima_regla
            , mp.fecha_nacimiento
            , mp.genero
            , mhe.cod_red
            , mhe.red
            , mhe.cod_mred
            , mhe.microred
            , mhe.provincia
            , mhe.distrito
            , mhe.cod_eess
            , nt.id_pais
            , concat(
                mp2.numero_documento
                , ' - '
                , mp2.nombres_personal
                , ' '
                , mp2.apellido_paterno_personal
                , ' '
                , mp2.apellido_materno_personal
            ) AS personal
            , concat(
                mr.numero_documento
                , ' - '
                , mr.nombres_registrador
                , ' '
                , mr.apellido_paterno_registrador
                , ' '
                , mr.apellido_materno_registrador
            ) as registrador
            ,
    ROW_NUMBER() OVER (PARTITION BY mp.id_tipo_documento, mp.numero_documento ORDER BY nt.anio, nt.fecha_atencion DESC ) AS rn
        FROM
            maestros.nominal_trama nt
            --INNER JOIN maestros.nominal_trama nt2 ON nt.id_cita= nt2.id_cita
        INNER JOIN maestros.maestro_paciente mp ON
            mp.id_paciente = nt.id_paciente
        LEFT JOIN maestros.eess_geresa_cusco mhe ON
            mhe.id_eess = nt.id_establecimiento
        LEFT JOIN maestros.maestro_his_tipo_doc mhtd ON
            mhtd.id_tipo_documento = mp.id_tipo_documento
        LEFT JOIN maestros.maestro_personal mp2 ON
            mp2.id_personal = nt.id_personal
        LEFT JOIN maestros.maestro_registrador mr ON
            mr.id_registrador = nt.id_registrador
        LEFT JOIN maestros.maestro_his_centro_poblado mhcp ON
            nt.id_centro_poblado = mhcp.id_centro_poblado
        WHERE
            (
                nt.codigo_item LIKE 'Z349_'
                    OR
                nt.codigo_item LIKE 'Z359_'
            )
            AND nt.anio = 2023
            --AND nt.mes = 2
    )AS t WHERE     t.rn = 1
  
""")
    return gestantes
def consulta_obs(conn):
    obs=conn.df("SELECT mo.id AS id_obs, mo.descripcion  FROM materno.maestro_observaciones mo ")  
    return obs

#consulta a la base de datos altitud de establecimiento 
def consulta_peso_edad(conn):
    peso_edad=conn.df("select * from oms_peso_edad ope")  
    return peso_edad

def consulta_talla_edad(conn):
    talla_edad=conn.df("select * from oms_talla_edad ote ")  
    return talla_edad

def consulta_peso_talla(conn):
    peso_talla=conn.df("select * from oms_peso_talla opt ")  
    return peso_talla

In [25]:
#menores=consulta_sql(conn)
def estructura_df(gestantes):

    gestantes = gestantes.drop(['rn'], axis=1)   
    gestantes['fecha_atencion'] = pd.to_datetime(gestantes['fecha_atencion'],format='%Y-%m-%d')
    gestantes['fecha_ultima_regla'] = pd.to_datetime(gestantes['fecha_ultima_regla'],format='%Y-%m-%d')
    gestantes = gestantes.sort_values(by=['fecha_atencion'], inplace=False)
    gestantes['peso'] = gestantes['peso'].fillna(0).apply(float)   
    gestantes['talla'] = gestantes['talla'].fillna(0).apply(float)
    gestantes['id_paciente'] = gestantes['id_paciente'].astype('int64', errors='raise') 

    # INDEXAR
    gestantes = gestantes.set_index(['abrev_tipo_doc','numero_documento'])
    # extraer el valosr maximo
    gestantes = gestantes.groupby(gestantes.index).tail(1)
    #menor.head(2)
    gestantes=gestantes.reset_index()   
    
    return gestantes

In [26]:
gestantes=consulta_sql(conn)
df2=estructura_df(gestantes)
df2.head(5)

d:\Irvin\Irvin\Python\Python-GERESA\modulos\gestante\../..\clases\bd\conexion2.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)


,abrev_tipo_doc,numero_documento,id_paciente,anio,mes,id_establecimiento,nombre_eess,edad_anio,edad_mes,edad_dia,fecha_atencion,lote,num_pag,num_reg,codigo_item,talla,peso,peso_pregestacional,fecha_ultima_regla,fecha_nacimiento,genero,cod_red,red,cod_mred,microred,provincia,distrito,cod_eess,id_pais,personal,registrador
0,DNI,72140430,280477752347,2023,1,2347,PISAC,27,328.1,9986,2023-01-01,A04,1,2,Z3593,154.1,79.9,NaN,NaT,1995-08-30,F,2,CUSCO NORTE,5,PISAC,CALCA,PISAC,2348,PER,40326391 - KATERINE JOSSIES CESPEDES CASTILLO,70818790 - JANET SANTA CRUZ AMASIFUEN
1,DNI,43211243,213172962479,2023,1,2479,PUCYURA,37,447.5,13622,2023-01-01,A02,5,1,Z3593,0.0,0.0,NaN,NaT,1985-09-15,F,3,LA CONVENCION,6,PUCYURA,LA CONVENCION,VILCABAMBA,2480,PER,02292359 - ENSUEÑO BRAVO HUACOTO,23965094 - GLICERIO CUEVA QUILLAHUAMAN
2,DNI,70751660,2702526036693,2023,1,36693,AYACCASI,28,341.7,10401,2023-01-01,Y03,1,2,Z3592,160.1,75.1,69.6,2022-08-02,1994-07-11,F,6,CHUMBIVILCAS,9,SANTO TOMAS,CHUMBIVILCAS,VELILLE,27011,PER,40471538 - YULY SILVERA ENCISO,73126866 - IDALIA SONCCO LAYME
3,DNI,60477836,253159859689,2023,1,9689,HUARACO,17,213.9,6510,2023-01-01,H05,4,2,Z3592,134.1,51.1,46.1,2022-07-18,2005-03-06,F,6,CHUMBIVILCAS,9,SANTO TOMAS,CHUMBIVILCAS,COLQUEMARCA,8913,PER,74350181 - JACKELIN SILVERA YERBA,74828344 - VERONICA ALVIS DAVILA
4,DNI,44659367,222042332288,2023,1,2288,HOSPITAL REGIONAL CUSCO,39,474.0,14428,2023-01-01,N59,14,10,Z3593,0.0,0.0,NaN,NaT,1983-07-02,F,0,HOSPITALES III,0,HOSPITAL,CUSCO,CUSCO,2289,PER,46588766 - JANINA MILAGROS MALAGA LOVATON,76248060 - JHON STIVENT BAÑON GOMEZ


In [35]:

df=df2

df['id_obs'] = 0
df.loc[df['talla'].isnull(), 'id_obs'] = 1
df.loc[df['peso'].isnull(), 'id_obs'] = 2
df.loc[df['fecha_ultima_regla'].isnull(), 'id_obs'] = 3
df.loc[df['peso_pregestacional'].isnull(), 'id_obs'] = 4
###obs=consulta_obs(conn)
##df = df.merge(obs, how='inner', on=['id_obs']) 

df['diagnostico'] = ''
df.loc[df['talla'].isnull(), 'diagnostico'] += 'Registro en blanco en talla. |'
df.loc[df['peso'].isnull(), 'diagnostico'] += 'Registro en blanco en peso. |'
df.loc[df['fecha_ultima_regla'].isnull(), 'diagnostico'] += 'Registro en blanco en fecha_ultima_regla. |'
df.loc[df['peso_pregestacional'].isnull(), 'diagnostico'] += 'Registro en blanco en peso_pregestacional. |'

df_observados = df[df['id_obs'] > 0]
df_observados.head()

,abrev_tipo_doc,numero_documento,id_paciente,anio,mes,id_establecimiento,nombre_eess,edad_anio,edad_mes,edad_dia,fecha_atencion,lote,num_pag,num_reg,codigo_item,talla,peso,peso_pregestacional,fecha_ultima_regla,fecha_nacimiento,genero,cod_red,red,cod_mred,microred,provincia,distrito,cod_eess,id_pais,personal,registrador,id_obs,diagnostico
0,DNI,72140430,280477752347,2023,1,2347,PISAC,27,328.1,9986,2023-01-01,A04,1,2,Z3593,154.1,79.9,NaN,NaT,1995-08-30,F,2,CUSCO NORTE,5,PISAC,CALCA,PISAC,2348,PER,40326391 - KATERINE JOSSIES CESPEDES CASTILLO,70818790 - JANET SANTA CRUZ AMASIFUEN,4,Registro en blanco en fecha_ultima_regla. |Reg...
1,DNI,43211243,213172962479,2023,1,2479,PUCYURA,37,447.5,13622,2023-01-01,A02,5,1,Z3593,0.0,0.0,NaN,NaT,1985-09-15,F,3,LA CONVENCION,6,PUCYURA,LA CONVENCION,VILCABAMBA,2480,PER,02292359 - ENSUEÑO BRAVO HUACOTO,23965094 - GLICERIO CUEVA QUILLAHUAMAN,4,Registro en blanco en fecha_ultima_regla. |Reg...
4,DNI,44659367,222042332288,2023,1,2288,HOSPITAL REGIONAL CUSCO,39,474.0,14428,2023-01-01,N59,14,10,Z3593,0.0,0.0,NaN,NaT,1983-07-02,F,0,HOSPITALES III,0,HOSPITAL,CUSCO,CUSCO,2289,PER,46588766 - JANINA MILAGROS MALAGA LOVATON,76248060 - JHON STIVENT BAÑON GOMEZ,4,Registro en blanco en fecha_ultima_regla. |Reg...
5,DNI,74567640,46085372519,2023,1,2519,HUANCARANI,16,201.1,6121,2023-01-01,A06,1,4,Z3593,144.0,60.0,NaN,NaT,2006-03-30,F,1,CUSCO SUR,6,PAUCARTAMBO,PAUCARTAMBO,HUANCARANI,2520,PER,41974147 - ANALI ROSMERY VENTURA FLORES,41433418 - BORIS OMAR PAREDES SALAS,4,Registro en blanco en fecha_ultima_regla. |Reg...
6,DNI,48153769,5379982307,2023,1,2307,ZARZUELA ALTA,30,367.4,11185,2023-01-01,A09,10,6,Z3593,153.8,79.3,NaN,NaT,1992-05-18,F,2,CUSCO NORTE,1,BELEMPAMPA,CUSCO,SANTIAGO,2308,PER,23990814 - ANITA ADELINA GOYZUETA REVILLA,43899116 - CINTHYA INGRID GUTIERREZ ALANOCA,4,Registro en blanco en fecha_ultima_regla. |Reg...


In [38]:
#pd.set_option('display.max_columns', None)
#conn.insert_df(df,'observaciones_his','materno')
conn.sqli(df_observados,'materno.observaciones_his')

'abrev_tipo_doc,numero_documento,id_paciente,anio,mes,id_establecimiento,nombre_eess,edad_anio,edad_mes,edad_dia,fecha_atencion,lote,num_pag,num_reg,codigo_item,talla,peso,peso_pregestacional,fecha_ultima_regla,fecha_nacimiento,genero,cod_red,red,cod_mred,microred,provincia,distrito,cod_eess,id_pais,personal,registrador,id_obs,diagnostico'